For data scientists, handling missing data is an important part of the data cleaning and model development process. Often times, real data contains multiple sparse fields or fields that are laden with bad values. In this post, we will discuss how to build models that can be used to impute missing or bad values in data.

Let’s get started!

For our purposes, we will be working with the wines dataset which can be found here.

To start, let’s read the data into a Pandas data frame:

In [1]:
import pandas as pd
df = pd.read_csv("winemag-data-130k-v2.csv")

Let’s take a random sample of 500 records from this data. This will help with speeding up model training and testing, though it can easily be modified by the reader:

In [3]:
import pandas as pd
df = pd.read_csv("winemag-data-130k-v2.csv").sample(n=500, random_state = 42)

Now, let’s print the info corresponding to our data which will give us an idea of which columns have missing values:

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 77718 to 101901
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             500 non-null    int64  
 1   country                500 non-null    object 
 2   description            500 non-null    object 
 3   designation            369 non-null    object 
 4   points                 500 non-null    int64  
 5   price                  471 non-null    float64
 6   province               500 non-null    object 
 7   region_1               424 non-null    object 
 8   region_2               188 non-null    object 
 9   taster_name            407 non-null    object 
 10  taster_twitter_handle  385 non-null    object 
 11  title                  500 non-null    object 
 12  variety                500 non-null    object 
 13  winery                 500 non-null    object 
dtypes: float64(1), int64(2), object(11)
memory usage: 5

Several columns have less than 500 non-null values, which correspond to missing values. First let’s consider building a model that imputes missing ‘price’ values using the ‘points’. To start, let’s print the correlation between ‘price’ and ‘points’:

In [5]:
print("Correlation: ", df['points'].corr(df['price']))

Correlation:  0.48057451325389017


We see that there is a weak positive correlation. Let’s build a linear regression model that uses ‘points’ to predict the ‘price’. First, let’s import the ‘LinearRegresssion’ module from ‘scikit-learn’:

In [6]:
from sklearn.linear_model import LinearRegression

Now, let’s split our data for training and testing. We’d like to be able to predict missing values, but we should use ground truth ‘price’ values to validate our predictions. Let’s filter out the missing values by selecting only positive price values:

In [7]:
import numpy as np 
df_filter = df[df['price'] > 0].copy()

Let’s also initialize lists we will use to store our predictions and actual values:

In [8]:
y_pred = []
y_true = []

We will use K-fold cross validation to validate our model. Let’s import the ‘KFolds’ module from ‘scikit-learn’. We will use 10 folds to validate our model:

In [24]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state = 42,shuffle=True)
for train_index, test_index in kf.split(df_filter):
    df_test = df_filter.iloc[test_index]
    df_train = df_filter.iloc[train_index]

We can now define our input and output:

In [25]:
for train_index, test_index in kf.split(df_filter):
    ...
    X_train = np.array(df_train['points']).reshape(-1, 1)     
    y_train = np.array(df_train['price']).reshape(-1, 1)
    X_test = np.array(df_test['points']).reshape(-1, 1)  
    y_test = np.array(df_test['price']).reshape(-1, 1)

And fit our linear regression model:

In [26]:
for train_index, test_index in kf.split(df_filter):
    ...
    model = LinearRegression()
    model.fit(X_train, y_train)

Now let’s generate and store our predictions:

In [27]:
for train_index, test_index in kf.split(df_filter):
    ...
    y_pred.append(model.predict(X_test)[0])
    y_true.append(y_test[0])

Now let’s evaluate the performance of our model. Let’s use mean squared error to evaluate the performance of our model:

In [28]:
from sklearn.metrics import mean_squared_error
print("Mean Square Error: ", mean_squared_error(y_true, y_pred))

Mean Square Error:  419.80600639139703


We see that the performance isn’t too great. We can improve this by training on prices bound by the mean price plus one standard deviation:

In [23]:
df_filter = df[df['price'] <= df['price'].mean() + df['price'].std() ].copy()
...
print("Mean Square Error: ", mean_squared_error(y_true, y_pred))

Mean Square Error:  544.3550140529626


While this significantly improves performance this comes at the price of not being able to accurately impute values for highly priced wines. Instead of using a regression model of a single feature to predict price, we can used tree base models, such as random forests, which can handle categorical and numerical variables. Let’s build a random forest regression model that uses ‘country’, ‘province’, ‘variety’, ‘winery’ and ‘points’ to predict wine ‘price’. First, let’s convert the categorical variables into categorical codes that can be handled by the random forests model:

In [31]:
df['country_cat'] = df['country'].astype('category')
df['country_cat'] = df['country_cat'].cat.codes
df['province_cat'] = df['province'].astype('category')
df['province_cat'] = df['province_cat'].cat.codes
df['winery_cat'] = df['winery'].astype('category')
df['winery_cat'] = df['winery_cat'].cat.codes
df['variety_cat'] = df['variety'].astype('category')
df['variety_cat'] = df['variety_cat'].cat.codes

Let’s increase the random sample size to 5000 :

In [32]:
df = pd.read_csv("winemag-data-130k-v2.csv").sample(n=5000, random_state = 42)

Next, let’s import the random forest regressor module from scikit-learn. Let’s also define the list of features we will use to train our model:

In [33]:
from sklearn.ensemble import RandomForestRegressor
features = ['points', 'country_cat', 'province_cat', 'winery_cat', 'variety_cat']

Let’s train our model using a random forest with 1000 estimators and a max depth of 1000. Let’s then generate predictions and append them to a new list:

In [34]:
for train_index, test_index in kf.split(df_filter):
    df_test = df_filter.iloc[test_index]
    df_train = df_filter.iloc[train_index]
    
    X_train = np.array(df_train[features])
    y_train = np.array(df_train['price'])
    X_test = np.array(df_test[features])
    y_test = np.array(df_test['price'])
    model = RandomForestRegressor(n_estimators = 1000, max_depth = 1000, random_state = 42)
    model.fit(X_train, y_train)
    y_pred_rf.append(model.predict(X_test)[0])
    y_true_rf.append(y_test[0])

KeyError: "['winery_cat', 'country_cat', 'province_cat', 'variety_cat'] not in index"